In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_val_score, cross_val_predict,KFold, StratifiedKFold
from catboost import CatBoostClassifier
import numpy as np

In [3]:
import pandas as pd
train=pd.read_csv('Training Data.csv')
test=pd.read_csv('Test Data.csv')

In [8]:
#train.drop('Id',axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

In [9]:
test.head()

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years
0,7393090,59,19,single,rented,no,Geologist,Malda,West Bengal,4,13
1,1215004,25,5,single,rented,no,Firefighter,Jalna,Maharashtra,5,10
2,8901342,50,12,single,rented,no,Lawyer,Thane,Maharashtra,9,14
3,1944421,49,9,married,rented,yes,Analyst,Latur,Maharashtra,3,12
4,13429,25,18,single,rented,yes,Comedian,Berhampore,West Bengal,13,11


In [10]:
#Remove duplicate rows from training set
train.drop_duplicates(keep='first',inplace = True)
train.duplicated().sum()

0

In [11]:
cat_cols = train.select_dtypes(include = 'object')
num_cols = train.select_dtypes(include=['int64','float64'])

In [12]:
combine = train.append(test)

In [13]:
import matplotlib.pyplot as plt

In [27]:
combine['married']=combine['married'].replace({'single':0,'married':1})

combine['house_ownership']=combine['house_ownership'].replace({'rented':1,'norent_noown':0,'owned':2})
combine['car_ownership']=combine['car_ownership'].replace({'yes':1,'no':0})

In [29]:
B_tech = [int(i in ["Mechanical engineer","Web designer","Software Developer","Design Engineer","Chemical engineer","Biomedical Engineer","Computer hardware engineer","Petroleum Engineer","Engineer","Civil engineer","Industrial Engineer","Mechanical engineer"]) for i in combine ["profession"]]
combine ["B_tech"] = pd.Series(B_tech)

public_service = [int(i in ["Civil servant","Firefighter","Politician","Police officer","Surveyor","Secretary","Magistrate","Lawyer","Army officer"]) for i in combine ["profession"]]
combine ["public_service"] = pd.Series(public_service)

design = [int(i in ["Architect","Designer","Artist","Design Engineer","Graphic Designer","Fashion Designer","Drafter"]) for i in combine ["profession"]]
combine ["design"] = pd.Series(design)

medicine = [int(i in ["Physician","Designer","Dentist","Microbiologist","Surgeon","Psychologist"]) for i in combine ["profession"]]
combine ["medicine"] = pd.Series(medicine)

finance = [int(i in ["Chartered Accountant","Economist","Financial Analyst","Statistician"]) for i in combine ["profession"]]
combine ["finance"] = pd.Series(finance)

self = [int(i in ["Technical writer","Librarian","Flight attendant","Air traffic controller","Hotel Manager","Comedian","Aviator","Official","Geologist","Chef","Scientist","Consultant","Technology specialist"]) for i in combine ["profession"]]
combine ["self"] = pd.Series(self)

#combine["incomeperprofession"] = combine.groupby(["income","profession"])["Id"].transform("count")
#combine["incomeperstate"] = combine.groupby(["income","state"])["Id"].transform("count")

In [32]:
combine["incomeperprofession"] = combine.groupby(["income","profession"])["state"].transform("count")
combine["incomepercity"] = combine.groupby(["income","city"])["state"].transform("count")

In [33]:
combine.head()

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,...,risk_flag,B_tech,public_service,design,medicine,finance,self,incomeperprofession,incomeperstate,incomepercity
0,1303835,23,3,0,1,0,Mechanical_engineer,Rewa,Madhya_Pradesh,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,4
1,7574516,40,10,0,1,0,Software_Developer,Parbhani,Maharashtra,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,4,4
2,3991815,66,4,1,1,0,Technical_writer,Alappuzha,Kerala,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,8,8
3,6256451,41,2,0,1,1,Software_Developer,Bhubaneswar,Odisha,2,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,7,7
4,5768871,47,11,0,1,0,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,6


In [66]:
df = combine

In [36]:
cat_col=['married','house_ownership', 'car_ownership', 'experience', 'current_job_years','profession','city','state']

In [68]:
train = df[df['risk_flag'].isnull()!= True]
test = df[df['risk_flag'].isnull()== True]

In [69]:
test.drop(['risk_flag'],axis=1,inplace=True)

C:\Users\DELL\anaconda3\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [70]:
X = train.drop(["risk_flag"], axis=1)
Y = train["risk_flag"]

In [67]:
from sklearn.preprocessing import LabelEncoder
x=['profession','city','state']
for i in x:
  le = LabelEncoder()
  df[i] = le.fit_transform(df[i])
  df[i]=df[i]+1

In [71]:
X.head()

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,B_tech,public_service,design,medicine,finance,self,incomeperprofession,incomeperstate,incomepercity
0,1303835,23,3,0,1,0,51,263,18,3,13,0.0,0.0,0.0,0.0,0.0,0.0,1,1,4
1,7574516,40,10,0,1,0,64,237,19,9,13,0.0,0.0,0.0,0.0,0.0,0.0,1,4,4
2,3991815,66,4,1,1,0,69,9,16,4,10,0.0,0.0,0.0,0.0,0.0,0.0,1,8,8
3,6256451,41,2,0,1,1,64,56,22,2,12,0.0,0.0,0.0,0.0,0.0,0.0,2,7,7
4,5768871,47,11,0,1,0,19,313,28,3,14,0.0,0.0,0.0,0.0,0.0,0.0,2,2,6


In [72]:
from math import sqrt 
from sklearn.metrics import roc_auc_score

In [73]:
oof_pred               = np.zeros((len(train),))
y_pred_final           = np.zeros((len(test),))
num_models             = 3

n_splits               = 20
error                  = []

kf=StratifiedKFold(n_splits=10,shuffle=True,random_state=294)
    
for i,(train_idx,val_idx) in enumerate(kf.split(X,Y)):    
    
    wghts                     = [0]*num_models
    test_roc_score            = []
    
    
    X_train, y_train = X.iloc[train_idx,:], Y.iloc[train_idx]

    X_val, y_val = X.iloc[val_idx, :], Y.iloc[val_idx]
    

    print('\nFold: {}\n'.format(i+1))

    model1 = CatBoostClassifier(learning_rate = 0.03,random_state=42,scale_pos_weight=7, custom_metric=['AUC'],task_type="CPU")
    model1.fit(X_train,y_train,cat_features=cat_col,eval_set=(X_val, y_val),early_stopping_rounds=30,verbose=100)
    testpred1 = model1.predict(X_val)
    test_roc_score.append(roc_auc_score(y_val, testpred1))
    print("Test ROC AUC for model 1: %.4f"%(roc_auc_score(y_val, testpred1)))
    
    model2 = CatBoostClassifier(learning_rate = 0.04,random_state=42,scale_pos_weight=7, custom_metric=['AUC'],task_type="CPU")
    model2.fit(X_train,y_train,cat_features=cat_col,eval_set=(X_val, y_val),early_stopping_rounds=40,verbose=100)
    testpred2 = model2.predict(X_val)
    test_roc_score.append(roc_auc_score(y_val, testpred2))
    print("Test ROC AUC for model 2: %.4f"%(roc_auc_score(y_val, testpred2)))
    
    model3 = CatBoostClassifier(learning_rate = 0.05,random_state=42,scale_pos_weight=7, custom_metric=['AUC'],task_type="CPU")
    model3.fit(X_train,y_train,cat_features=cat_col,eval_set=(X_val, y_val),early_stopping_rounds=20,verbose=100)
    testpred3 = model3.predict(X_val)
    test_roc_score.append(roc_auc_score(y_val, testpred3))
    print("Test ROC AUC for model 3: %.4f"%(roc_auc_score(y_val, testpred3)))
    
    wghts              = np.exp(-1000*np.array(test_roc_score/sum(test_roc_score)))
    wghts              = wghts/sum(wghts)
    
    val_pred           = wghts[0]*testpred1+wghts[1]*testpred2 +wghts[2]*testpred3
    print('validation roc_auc_score fold-',i+1,': ',roc_auc_score(y_val, val_pred))
    
    oof_pred[val_idx]  = val_pred
    y_pred_final += (wghts[0]*model1.predict(test)+wghts[1]*model2.predict(test)+wghts[2]*model3.predict(test)/(n_splits)) 
    


Fold: 1

0:	learn: 0.6867471	test: 0.6869250	best: 0.6869250 (0)	total: 255ms	remaining: 4m 14s
100:	learn: 0.6108200	test: 0.6095235	best: 0.6095235 (100)	total: 14.9s	remaining: 2m 12s
200:	learn: 0.5956406	test: 0.5998098	best: 0.5989266 (192)	total: 23.9s	remaining: 1m 35s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.598926624
bestIteration = 192

Shrink model to first 193 iterations.
Test ROC AUC for model 1: 0.5682
0:	learn: 0.6847095	test: 0.6849440	best: 0.6849440 (0)	total: 100ms	remaining: 1m 39s
100:	learn: 0.6060589	test: 0.6102274	best: 0.6098259 (92)	total: 8.29s	remaining: 1m 13s
200:	learn: 0.5917740	test: 0.5955433	best: 0.5955433 (200)	total: 17.5s	remaining: 1m 9s
300:	learn: 0.5827678	test: 0.5925905	best: 0.5911612 (281)	total: 27s	remaining: 1m 2s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.5911612418
bestIteration = 281

Shrink model to first 282 iterations.
Test ROC AUC for model 2: 0.5798
0:	learn: 0.6827195	test:

100:	learn: 0.6096797	test: 0.6116952	best: 0.6116952 (100)	total: 8.85s	remaining: 1m 18s
200:	learn: 0.5963862	test: 0.5998376	best: 0.5994087 (198)	total: 34.4s	remaining: 2m 16s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.5994086517
bestIteration = 198

Shrink model to first 199 iterations.
Test ROC AUC for model 1: 0.5665
0:	learn: 0.6904454	test: 0.6904469	best: 0.6904469 (0)	total: 33.8ms	remaining: 33.8s
100:	learn: 0.6055412	test: 0.6129543	best: 0.6120306 (98)	total: 9.1s	remaining: 1m 21s
200:	learn: 0.5941067	test: 0.5977416	best: 0.5977416 (199)	total: 19.4s	remaining: 1m 17s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.5930408279
bestIteration = 245

Shrink model to first 246 iterations.
Test ROC AUC for model 2: 0.5756
0:	learn: 0.6897876	test: 0.6897895	best: 0.6897895 (0)	total: 21.2ms	remaining: 21.1s
100:	learn: 0.6008890	test: 0.6004749	best: 0.6004695 (95)	total: 6.14s	remaining: 54.7s
Stopped by overfitting detector  

NameError: name 'y_pred_final' is not defined